<a href="https://colab.research.google.com/github/alex-jk/datakit-smallholder-farmers-fall-2025/blob/main/Prep%20Challenge-%20Translation/alex_j_farmers_data_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Alex-J Farmer Survey Data: Translation Pipeline

This notebook processes a dataset of survey questions and responses from smallholder farmers. The primary goal is to translate non-English text (specifically Swahili) into English to create a unified dataset for further analysis.

### Workflow Overview:
1.  **Setup & Data Loading:** The environment is configured by cloning the project repository and mounting Google Drive. The large survey CSV is loaded into a `duckdb` in-memory database and converted to the efficient Parquet format for faster querying.
2.  **Exploratory Data Analysis (EDA):** Initial analysis is performed to understand the dataset's structure, size, language distribution, and overall data quality.
3.  **Translation with a Hugging Face Model:** A free, open-source Swahili-to-English translation model from the Hugging Face Hub (`Bildad/Swahili-English_Translation`) is used to test the translation process on a small sample of the data.
4.  **Sampling for Google Cloud Translation:** To manage costs for a higher-quality translation service, the notebook:
    *   Estimates the potential cost of using the Google Cloud Translation API.
    *   Implements a stratified sampling algorithm to create a representative 8,000-question sample, ensuring proportional representation from all question topics.
    *   Exports this curated sample, preparing it for batch translation via the Google Cloud API.

<h3><font color="#0b3d91">Clone the repo</font></h3>

In [ ]:
import os
import pandas as pd
import numpy as np
import math
import time
from google.colab import drive
from tqdm.auto import tqdm

# Always work from /content when checking / cloning the repo
root_dir = "/content"
repo_name = "datakit-smallholder-farmers-fall-2025"
repo_dir = os.path.join(root_dir, repo_name)

# Go to /content first
%cd /content

# Clone only if the repo directory does NOT exist in /content
if not os.path.isdir(repo_dir):
    !git clone https://github.com/alex-jk/datakit-smallholder-farmers-fall-2025.git

# Now cd into the repo, then into the subfolder
%cd {repo_dir}
%cd "Prep Challenge- Translation"

<h3><font color="#0b3d91">Load the farmers survey dataset</font></h3>

In [ ]:
drive.mount('/content/drive')

export_input_to_parquet = False
run_sample_query = False
perform_google_translate = False

!pip install duckdb -q

In [ ]:
import duckdb

csv_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_survey_dataset.csv"
parquet_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_survey_dataset.parquet"

con = duckdb.connect()

In [ ]:
if export_input_to_parquet:

  con.execute(f"""
      COPY (
          SELECT *
          FROM read_csv_auto('{csv_path}', sample_size=-1)
      )
      TO '{parquet_path}'
      (FORMAT PARQUET);
  """)

**Connect to the DB**

In [ ]:
parquet_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_survey_dataset.parquet"

con.execute(f"""
    CREATE OR REPLACE VIEW farmers AS
    SELECT *
    FROM read_parquet('{parquet_path}')
""")

In [ ]:
# Row count
row_count_df = con.execute("SELECT COUNT(*) AS n_rows FROM farmers").df()
n_rows = int(row_count_df.loc[0, "n_rows"])

# Column info
cols_info = con.execute("PRAGMA table_info('farmers')").df()
n_cols = cols_info.shape[0]
col_names = cols_info["name"].tolist()

print(f"DF row count: {n_rows}")
print(f"DF column count: {n_cols}")
print("Column names:")
print(col_names)

# Peek at data
con.execute("SELECT * FROM farmers LIMIT 5").df()

**Check unique question counts**

In [ ]:
print("Waiting 5 seconds before running query...")
time.sleep(5)

questions_counts = con.execute("""
    SELECT
        question_id,
        question_topic,
        question_content,
        question_language,
        COUNT(*) AS n_responses
    FROM farmers
    GROUP BY
        question_id,
        question_topic,
        question_content,
        question_language
    ORDER BY
        n_responses DESC
""").df()

print(questions_counts.shape)
# display(questions_counts)

**Check if each question ID corresponds to a unique question**

In [ ]:
# All question_ids where there is more than 1 distinct question_content
qid_conflicts = con.execute("""
    SELECT
        question_id,
        COUNT(DISTINCT question_content) AS n_question_contents
    FROM farmers
    GROUP BY question_id
    HAVING COUNT(DISTINCT question_content) > 1
    ORDER BY n_question_contents DESC
""").df()

qid_conflicts

**Check questions and responses**

In [ ]:
pd.set_option("display.max_colwidth", None)

qid_counts_by_lang = con.execute("""
    SELECT
        question_language,
        COUNT(DISTINCT question_id) AS n_unique_question_ids
    FROM farmers
    GROUP BY question_language
    ORDER BY question_language
""").df()

qid_counts_by_lang["n_unique_question_ids_fmt"] = (
    qid_counts_by_lang["n_unique_question_ids"].astype(int).map("{:,}".format)
)

display(qid_counts_by_lang)

select_lang = "swa"  # or "swa", "lug", ...

sample_lang = con.execute("""
    SELECT
        question_id,
        question_topic,
        question_content,
        response_content,
        question_language
    FROM farmers
    WHERE question_language = ?
      AND question_content IS NOT NULL
      AND response_content IS NOT NULL
    ORDER BY random()
    LIMIT 20
""", [select_lang]).df()

print(f"\n-------Questions Sample {select_lang}------\n")
display(sample_lang)

<h3><font color="#0b3d91">Translation</font></h3>

In [ ]:
!pip install huggingface_hub -q

from huggingface_hub import HfFolder

# Remove any stored token so transformers uses anonymous access
HfFolder.delete_token()

In [ ]:
!pip install transformers sentencepiece accelerate -q

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Bildad/Swahili-English_Translation"  # Swahili <-> English

print("Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

print("Model loaded.")
print("Device:", device)

# --- quick sanity check for inference ---
try:
    test_texts = ["Dawa ya minyoo ya kuku ni gani?"]
    enc = tokenizer(
        test_texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=64,
    ).to(device)

    with torch.no_grad():
        out = model.generate(**enc, max_length=64, num_beams=1)

    test_translation = tokenizer.batch_decode(out, skip_special_tokens=True)
    print("Sanity check translation OK:")
    print("  Input :", test_texts[0])
    print("  Output:", test_translation[0])

except Exception as e:
    print("Sanity check FAILED, model not ready for inference.")
    print("Error:", repr(e))

In [ ]:
def translate_batch(texts, max_length=128):
    """Translate a list of Swahili strings to English."""
    texts = [t if isinstance(t, str) else "" for t in texts]

    enc = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length,
    ).to(device)

    with torch.no_grad():
        out = model.generate(**enc, max_length=max_length)

    return tokenizer.batch_decode(out, skip_special_tokens=True)

**Test translator on a sample if questions**

In [ ]:
# Translate the question_content column with a progress bar
texts = sample_lang["question_content"].tolist()

batch_size = 32  # or 64/128 depending on how heavy the model is
all_translations = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i + batch_size]
    batch_translations = translate_batch(batch)
    all_translations.extend(batch_translations)

# Attach translations back to the DataFrame
sample_lang["question_content_en"] = all_translations

# Display original + translation
sample_lang[[
    "question_id",
    "question_topic",
    "question_content",
    "question_content_en",
]]

<h4><font color="#0b3d91"><b>Calculate Google API translation</b></font></h4>

In [ ]:
# --- 1. Helper: estimate cost given a list/Series of texts ---
def estimate_translate_cost(texts, free_chars=500_000, price_per_million=20.0):
    """
    texts: iterable of strings (each element = what you'd send as *one* request to Google)
    free_chars: monthly free tier characters (Google Cloud Translation)
    price_per_million: USD per 1,000,000 billable characters
    """
    # Ensure strings, ignore None/NaN
    lengths = [len(t) for t in texts if isinstance(t, str)]
    total_chars = sum(lengths)

    free_used = min(total_chars, free_chars)
    billable_chars = max(0, total_chars - free_chars)
    cost_usd = billable_chars / 1_000_000 * price_per_million

    avg_chars_per_row = (total_chars / len(lengths)) if lengths else 0
    rows_free = int(free_chars // avg_chars_per_row) if avg_chars_per_row > 0 else 0

    return {
        "n_rows": len(lengths),
        "total_chars": total_chars,
        "avg_chars_per_row": avg_chars_per_row,
        "free_chars_available": free_chars,
        "free_chars_used": free_used,
        "billable_chars": billable_chars,
        "estimated_cost_usd": cost_usd,
        "approx_rows_free_at_this_avg": rows_free,
    }

# --- 2. Pull 7K rows from your actual farmers data ---
select_lang = "swa"  # or "swa", "lug", ...

df_7k = con.execute("""
    SELECT
        question_id,
        question_content,
        response_content,
        question_language
    FROM farmers
    WHERE question_language = ?
       AND question_content IS NOT NULL
       AND response_content IS NOT NULL
    ORDER BY question_id
    LIMIT 7000
""", [select_lang]).df()

print("Sample shape:", df_7k.shape)

# Combine question + response into what you'd actually send to Google as one string
combined_7k = (
    df_7k["question_content"].fillna("")
    + "\n"
    + df_7k["response_content"].fillna("")
)

# --- 3. Estimate cost for THESE EXACT 7K ROWS ---
result_7k = estimate_translate_cost(combined_7k)

print("Rows:", result_7k["n_rows"])
print("Total characters:", result_7k["total_chars"])
print("Average chars per row:", round(result_7k["avg_chars_per_row"], 1))
print("Free chars available:", result_7k["free_chars_available"])
print("Free chars used in this 7K:", result_7k["free_chars_used"])
print("Billable chars (beyond free):", result_7k["billable_chars"])
print("Estimated cost for these 7K (USD):", round(result_7k["estimated_cost_usd"], 2))
print("At this average length, approx rows you could translate for free:",
      result_7k["approx_rows_free_at_this_avg"])

combined_7k = (
    df_7k["question_content"].fillna("")
    + "\n"
    + df_7k["response_content"].fillna("")
)

total_chars = combined_7k.str.len().sum()
avg_chars = combined_7k.str.len().mean()

print("Total chars:", total_chars)
print("Avg chars/row:", avg_chars)

<h4><font color="#0b3d91"><b>Questions selection for Google translation</b></font></h4>
This code creates a view of unique <b>Q&A pairs per question</b> (for a chosen language), picks one primary topic per question, and counts how many questions fall under each topic.<br>It then designs a sampling plan for exactly <b>N</b> questions: every topic gets at least one row and the remaining rows are allocated proportionally to topic frequency, with a final adjustment so the total sample size is exactly <b>N</b> and all topics are represented.

In [ ]:
target_total = 8000

con.execute(f"""
    CREATE OR REPLACE VIEW swa_unique_questions AS
    SELECT
        question_id,
        -- choose ONE primary topic per question for sampling
        MIN(question_topic) AS primary_topic,
        -- all topics for information
        STRING_AGG(DISTINCT question_topic, ',') AS all_topics,
        -- representative texts
        MAX(question_content)  AS question_content,
        MAX(response_content)  AS response_content,
        question_language
    FROM farmers
    WHERE question_language = '{select_lang}'
      AND question_content IS NOT NULL
      AND response_content IS NOT NULL
    GROUP BY
        question_id,
        question_language
""")

topic_counts = con.execute("""
    SELECT
        primary_topic AS question_topic,
        COUNT(*) AS n_rows
    FROM swa_unique_questions
    GROUP BY primary_topic
""").df()

n_topics_total = topic_counts.shape[0]

topic_counts["base"] = 1
remaining = target_total - n_topics_total
if remaining <= 0:
    raise ValueError("target_total too small to give every topic at least 1 row")

weights = topic_counts["n_rows"] / topic_counts["n_rows"].sum()
topic_counts["extra"] = (weights * remaining).round().astype(int)
topic_counts["sample_n"] = topic_counts["base"] + topic_counts["extra"]

diff = target_total - topic_counts["sample_n"].sum()
if diff > 0:
    idx = topic_counts["n_rows"].sort_values(ascending=False).index[:diff]
    topic_counts.loc[idx, "sample_n"] += 1
elif diff < 0:
    candidates = topic_counts[topic_counts["sample_n"] > 1].sort_values(
        "sample_n", ascending=False
    ).index[: -diff]
    topic_counts.loc[candidates, "sample_n"] -= 1

print("Final planned total:", topic_counts["sample_n"].sum())
print("Total topics in data:", n_topics_total)
print("Topics represented in sample:", (topic_counts["sample_n"] > 0).sum())

<h4><font color="#0b3d91"><b>Select final questions for the sample</b></font></h4>
This cell builds a view <b>`swa_unique_questions`</b> of unique questions per <b>`question_id`</b> and language, choosing one primary topic per question, aggregating all topics, and keeping representative question/response text. It then counts how many questions fall under each primary topic and designs a sampling plan for a fixed total of <b>N</b> questions. <br>Every topic is guaranteed at least one question, and the remaining sample slots are distributed proportionally to topic frequency, with a final adjustment step to ensure the total sample size is exactly <b>N</b>. <br>It prints the planned total sample size, the number of topics in the data, and how many topics are represented in the sample.


In [ ]:
all_samples = []

if run_sample_query:
    for _, row in topic_counts.iterrows():
        topic = row["question_topic"]      # this is primary_topic from topic_counts
        k = int(row["sample_n"])
        if k <= 0:
            continue

        if pd.isna(topic):  # topic is NULL / None
            df_topic = con.execute("""
                SELECT
                    question_id,
                    primary_topic AS question_topic,
                    all_topics,
                    question_content,
                    response_content,
                    question_language
                FROM swa_unique_questions
                WHERE primary_topic IS NULL
                ORDER BY hash(question_id)         -- deterministic
                LIMIT ?
            """, [k]).df()
        else:  # normal non-null topic
            df_topic = con.execute("""
                SELECT
                    question_id,
                    primary_topic AS question_topic,
                    all_topics,
                    question_content,
                    response_content,
                    question_language
                FROM swa_unique_questions
                WHERE primary_topic = ?
                ORDER BY hash(question_id)
                LIMIT ?
            """, [topic, k]).df()

        all_samples.append(df_topic)

    df_sample = pd.concat(all_samples, ignore_index=True)
    print("Initial sample size (rows):", df_sample.shape[0])
    print("Unique question_ids:", df_sample["question_id"].nunique())

    # If slightly above target_total, trim once with fixed seed
    if df_sample.shape[0] > target_total:
        df_sample = df_sample.sample(n=target_total, random_state=42).reset_index(drop=True)

    print("Final sample size (rows):", df_sample.shape[0])
    print("Final unique question_ids:", df_sample["question_id"].nunique())

In [ ]:
if run_sample_query:
  sample_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_swa_sample.parquet"

  df_sample = df_sample.drop_duplicates().reset_index(drop=True)
  print(f"\nDF SAMPLE num rows: {len(df_sample.index)}")

  df_sample.to_parquet(sample_path, index=False)
  print("Saved sample to:", sample_path)

<h4><font color="#0b3d91"><b>Translate sample questions</b></font></h4>
<b>Import DF sample</b>

In [ ]:
sample_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_swa_sample.parquet"

df_sample = pd.read_parquet(sample_path)
print(df_sample.shape)

n_topics_sample = df_sample["question_topic"].nunique()
print("Unique topics in sample:", n_topics_sample)
print(f"\nNumber of unique question IDs in the sample: {len(df_sample['question_id'].unique())}")
print(f"\nNumber of question content in the sample: {len(df_sample['question_content'].unique())}")

df_sample.head()

In [ ]:
!pip install -q google-cloud-translate

from google.cloud import translate_v2 as translate

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
creds_path = os.path.join(os.getcwd(), "farmers-survey-translation-key.json")
print("Using credentials file:", creds_path)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = creds_path

translate_client = translate.Client()

# sanity check
test = translate_client.translate(
    "Dawa ya minyoo ya kuku ni gani?",
    source_language="sw",
    target_language="en",
    format_="text",
)
print("Test translation:", test["translatedText"])

In [ ]:
def translate_list(texts, source="sw", target="en", batch_size=128):
    """
    Translate a list/Series of strings with Google Cloud Translation.
    Returns a list of translated strings in the same order.
    """
    texts = list(texts)
    out = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
        batch = [t if isinstance(t, str) else "" for t in texts[i:i+batch_size]]
        if not batch:
            continue

        result = translate_client.translate(
            batch,
            source_language=source,
            target_language=target,
            format_="text",
        )
        out.extend([r["translatedText"] for r in result])

    return out

In [ ]:
if perform_google_translate:
  # Translate question_content
  q_texts = df_sample["question_content"]
  q_en = translate_list(q_texts, source="sw", target="en", batch_size=128)
  df_sample["question_content_google_en"] = q_en

  # Quick check
  df_sample[[
      "question_id",
      "question_content",
      "question_content_google_en"
  ]].head(25)

  len_q_texts = len(df_sample["question_content"])
  len_q_en = len(q_en)

  print("Rows in df_sample:", len_q_texts)
  print("Translations returned:", len_q_en)

  print("Missing translations in column:",
        df_sample["question_content_google_en"].isna().sum())

In [ ]:
if perform_google_translate:
  out_parquet = "/content/drive/MyDrive/DataKind Farmers Project/swa_unique_q_8k_google_translated.parquet"

  # save Parquet
  df_sample.to_parquet(out_parquet, index=False)
  print("Saved Parquet to:", out_parquet)